## Python Processing for CHIR Data ##

In [3]:
# set-up steps
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import time
import os

warnings.filterwarnings("ignore")

# data loading

CHIR_df = pd.read_csv("CHIR_data.csv")

In [4]:
# setting up a data check for potential duplicates

# Typology for duplicates (pending confirmation)
dupe_code_list = {"cross_state_duplicate", "document_type_duplicate", "non_duplicate", "payer_duplicate","true_duplicate"}

# Converting the policy identifier column to a more useful format (numeric to string)
CHIR_df['Policy Identifier'] = CHIR_df['Policy Identifier'].astype(str)

# Creating a duplicate flagger function for this data stream
# This flags duplicates in the DataFrame based on Policy Identifier, States, Document Type, and Payer.

def flag_duplicates(CHIR_df):

    # Sort the DataFrame to ensure consistent grouping for duplicates
    CHIR_df_sorted = CHIR_df.sort_values(by=["Policy Identifier", "States", "Document Type", "Payer"])

    # Initialize the 'duplicate_flagger' column with a default value
    CHIR_df_sorted["duplicate_flagger"] = "non_duplicate"

    # Identify where there are the same 'Policy Identifier' but different 'States'.
    cross_state_dupes = CHIR_df_sorted.groupby("Policy Identifier").filter(
        lambda x: x["States"].nunique() > 1
    )
    CHIR_df_sorted.loc[cross_state_dupes.index, "duplicate_flagger"] = "cross_state_duplicate"
    

    # Identify where there are the same 'Policy Identifier' and 'States' but different 'Document Type'.
    doc_type_dupes_mask = (
        (CHIR_df_sorted.duplicated(subset=["Policy Identifier", "States"], keep=False))
        & (
            ~CHIR_df_sorted.duplicated(
                subset=["Policy Identifier", "States", "Document Type"], keep=False
            )
        )
        & (CHIR_df_sorted["duplicate_flagger"] == "non_duplicate")
    )
    CHIR_df_sorted.loc[doc_type_dupes_mask, "duplicate_flagger"] = "document_type_duplicate"
    


    # Identify where there are the same 'Policy Identifier' and 'States' but different 'Payer'.
    payer_dupes_mask = (
        (CHIR_df_sorted.duplicated(subset=["Policy Identifier", "States"], keep=False))
        & (
            ~CHIR_df_sorted.duplicated(
                subset=["Policy Identifier", "States", "Payer"], keep=False
            )
        )
        & (CHIR_df_sorted["duplicate_flagger"] == "non_duplicate")
    )
    CHIR_df_sorted.loc[payer_dupes_mask, "duplicate_flagger"] = "payer_duplicate"
    

    # Identifying true duplicates, where all these factors are identical
    true_dupes_mask = (
        CHIR_df_sorted.duplicated(
            subset=["Policy Identifier", "States", "Document Type", "Payer"], keep=False
        )
        & (CHIR_df_sorted["duplicate_flagger"] == "non_duplicate")
    )
    CHIR_df_sorted.loc[true_dupes_mask, "duplicate_flagger"] = "true_duplicate"
    

    return CHIR_df_sorted

# Running the function on the CHIR data
dupecheck_CHIR_df = flag_duplicates(CHIR_df.copy())

# Exporting the modified version
output_path = "derived_data/CHIR_processed_for_Airtable.csv"
dupecheck_CHIR_df.to_csv(output_path, index= False)

print(f"CHIR Processed Data successfully exported to: {output_path}")

CHIR Processed Data successfully exported to: derived_data/CHIR_processed_for_Airtable.csv


In [6]:
# CGM grouping exercise
# This categorizes CGM product coverage based on if they are 'Dexcom' and 'FreeStyle'


def categorize_cgm_coverage(df: pd.DataFrame, column_name: str) -> pd.DataFrame:

    # Create an empty list to store the labels for the new column
    cgm_grouping_labels = []

    # Iterate through each cell in the specified column
    for cell_value in df[column_name]:
        # Initialize flags for Dexcom and FreeStyle presence
        has_dexcom = False
        has_freestyle = False

        # Handle missing or empty values first
        if pd.isna(cell_value) or str(cell_value).strip() == '':
            cgm_grouping_labels.append("No coverage")
            continue # Move to the next cell

        # Convert the cell value to a string and then to lowercase for case-insensitive matching
        # Split the string by commas to get individual product names
        products = str(cell_value).lower().split(',')

        # Check for the presence of 'dexcom' and 'freestyle' in any of the product names
        for product in products:
            if "dexcom" in product:
                has_dexcom = True
            if "freestyle" in product:
                has_freestyle = True

        # Apply the labeling logic based on the flags
        if has_dexcom and has_freestyle:
            cgm_grouping_labels.append("both Dexcom and FreeStyle")
        elif has_dexcom:
            cgm_grouping_labels.append("Dexcom only")
        elif has_freestyle:
            cgm_grouping_labels.append("FreeStyle only")
        else:
            # If neither Dexcom nor FreeStyle are found, but the cell was not empty,
            # it means other products are covered.
            cgm_grouping_labels.append("Others covered")

    # Assign the generated labels to the new 'CGM_grouping' column in the DataFrame
    df['CGM_grouping'] = cgm_grouping_labels

    return df

# test run
cat_CHIR_df = categorize_cgm_coverage(dupecheck_CHIR_df.copy(), 'CGMs Covered')

# Exporting the modified version
output_path = "derived_data/cat_CHIR_processed_for_Airtable.csv"
cat_CHIR_df.to_csv(output_path, index= False)

print(f"CHIR Categorized CGM Data successfully exported to: {output_path}")

CHIR Categorized CGM Data successfully exported to: derived_data/cat_CHIR_processed_for_Airtable.csv


In [16]:
# grouping for implanted

# writing a dictionary based on the table
device_implant_dict = {
    "Eversense E3 rtCGM": "implanted",
    "Eversense 365": "implanted",
    "Medtronic Guardian 3 rtCGM": "non-implanted",
    "Medtronic Guardian 4 rtCGM": "non-implanted",
    "Medtronic Simplera rtCGM": "non-implanted",
    "Medtronic Simplera Sync rtCGM": "non-implanted",
    "Dexcom G6 rtCGM": "non-implanted",
    "Dexcom G7 rtCGM": "non-implanted",
    "Abbott FreeStyle Libre 2 Plus rtCGM": "non-implanted",
    "Abbott FreeStyle Libre 3 Plus rtCGM": "non-implanted",
    "Abbott FreeStyle Libre 2 isCGM": "non-implanted",
    "Abbott FreeStyle Libre 14 isCGM": "non-implanted",
    "Unspecified - transcutaneous rtCGM": "non-implanted",
    "Unspecified - transcutaneous isCGM": "non-implanted",
    "Unspecified - implantable rtCGM": "implanted",
    "Unspecified": "unknown"
}

# iterating through our data (with a copy for safety)
implanted_CHIR_df = cat_CHIR_df.dropna(subset=['CHIR Review Fields Last Modified']).copy()

# building an iterating function to run through the data
def categorize_implanted_type(df: pd.DataFrame, column_name: str) -> pd.DataFrame:

    # create an empty list to store the labels for the new column
    implanted_type_labels = []

    # iterate through each cell in the specified column
    for cell_value in df[column_name]:
        # handling missing or empty values
        if pd.isna(cell_value) or str(cell_value).strip() == '':
            implanted_type_labels.append("no devices")
            continue

        # separating multi-device coverage by commas
        products_in_cell = str(cell_value).split(',')
        
        has_implanted_device = False
        has_non_implanted_device = False
        
        # check in case any unknowns (or spelling variations)
        found_any_known_device_in_cell = False 

        # check each product in the cell against the dictionary
        for product_in_cell in products_in_cell:
            for mapped_device_name, status in device_implant_dict.items():
                # check for partial matches: if the mapped name is in the product string, or vice-versa
                if mapped_device_name in product_in_cell or product_in_cell in mapped_device_name:
                    found_any_known_device_in_cell = True
                    if status == "implanted":
                        has_implanted_device = True
                    elif status == "non-implanted":
                        has_non_implanted_device = True
                    break

        # applying the labeling logic based on the flags after checking all products in the cell
        if has_implanted_device and has_non_implanted_device:
            implanted_type_labels.append("both implanted and non-implanted")
        elif has_implanted_device:
            implanted_type_labels.append("implanted only")
        elif has_non_implanted_device:
            implanted_type_labels.append("non-implanted only")
        else:
            implanted_type_labels.append("unknown device type") 

    # assign the generated labels to the new 'Implanted_Type' column
    df['Implanted_Type'] = implanted_type_labels

    return df

# test run with our copied data

implanted_CHIR_df = categorize_implanted_type(implanted_CHIR_df, 'CGMs Covered')

# Exporting the modified version
output_path = "derived_data/implanted_CHIR_processed_for_Airtable.csv"
implanted_CHIR_df.to_csv(output_path, index= False)

print(f"CHIR Implanted/non-implanted CGM Data successfully exported to: {output_path}")

CHIR Implanted/non-implanted CGM Data successfully exported to: derived_data/implanted_CHIR_processed_for_Airtable.csv
